In [1]:
import cv2
import socket
import time

# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open a video strea
cap = cv2.VideoCapture('192.168.1.134:81')

# Define the Wi-Fi sending function
def send_number_over_wifi():
    HOST = '192.168.1.134'  # Replace with the IP address of the device receiving the number
    PORT = 80  # Replace with the port number of the receiving device
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT)) #ip adres en poort invullen (udp of tcp vermoed ik)
        s.sendall(b'300')

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    ret, frame = cap.read()
    time.sleep(1)
    print(cap.read())
    if frame is None:
        continue
    
    
    # Apply the face detection model to the frame
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

    # Draw bounding boxes around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Send the number 300 over Wi-Fi
        send_number_over_wifi()

    # Display the output
    cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video stream and close the window
cap.release()
cv2.destroyAllWindows()

(False, None)
(False, None)


KeyboardInterrupt: 

In [28]:
import cv2
import socket
import time
import numpy as np
from everywhereml.data import ImageDataset
from everywhereml.data.collect import MjpegCollector


# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open a video stream from the URL
url = "http://192.168.1.134"
video_stream = cv2.VideoCapture(url)

# Define the Wi-Fi sending function
def send_number_over_wifi():
    HOST = '192.168.1.134'  # Replace with the IP address of the device receiving the number
    PORT = 80  # Replace with the port number of the receiving device
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT)) #ip adres en poort invullen (udp of tcp vermoed ik)
        s.sendall(b'300')

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    mjpeg_collector = MjpegCollector(address='http://192.168.1.134')
    print(dir(mjpeg_collector))
    frame_data_list = mjpeg_collector.collect_by_samples(5)
    frame_data = b"".join(frame_data_list)
    frame = cv2.imdecode(np.frombuffer(frame_data, dtype=np.uint8), cv2.IMREAD_COLOR)
    time.sleep(1)
    print(mjpeg_collector)
    if frame is None:
        continue
   
    
    
    # Apply the face detection model to the frame
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

    # Draw bounding boxes around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Send the number 300 over Wi-Fi
        send_number_over_wifi()

    # Display the output
    cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video stream and close the window
video_stream.release()
cv2.destroyAllWindows()


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_collect', 'address', 'collect_by_duration', 'collect_by_samples', 'collect_many_classes', 'max_fps']


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:798: error: (-215:Assertion failed) !buf.empty() in function 'cv::imdecode_'


In [19]:
import cv2
import urllib.request
import numpy as np

stream_url = 'http://192.168.1.134:81'  # Replace with your stream URL

# Open the video stream
stream = urllib.request.urlopen(stream_url)

# Create a VideoCapture object to read from the stream
video_capture = cv2.VideoCapture()

# Loop over the frames from the video stream
while True:
    # Read the next frame from the video stream
    ret, frame = video_capture.read()
    print(ret)
    # If there was an error reading the frame, break out of the loop
    if not ret:
        break

    # Display the frame
    cv2.imshow('Frame', frame)

    # Wait for a key press
    key = cv2.waitKey(1)

    # If the 'q' key was pressed, break out of the loop
    if key == ord('q'):
        break

# Release the resources
video_capture.release()
cv2.destroyAllWindows()


False


In [36]:
import cv2
import urllib.request
import requests
import numpy as np
# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Define the ESP32 Wrover's IP address and port number
url = 'http://192.168.1.134:81'

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        print(a, b)
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        #print(a,b)
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            # Apply the face detection model to the frame
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

            # Draw bounding boxes around the detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                requests.get('http://192.168.1.134/signal?300')
            # Display the output
            cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]


# Release the video stream and close the window
cv2.destroyAllWindows()


50 1627
1752 1224
1353 824
954 405
555 11
156 23
1858 1307
1459 916
1060 495
661 99
262 19
1964 1389
1565 992
1166 578
767 191
368 62
2070 1498
1671 1090
1272 696
873 296
474 16
75 11
1777 1213
1378 819
979 431
580 32
181 48
1883 1341
1484 960
1085 568
686 171
287 44
1989 1519
1590 1119
1191 717
792 333
393 15
2095 1638
1696 1263
1297 865
898 464
499 62
100 0
1802 1374
1403 968
1004 575
605 176
206 73
1908 1468
1509 1068
1110 673
711 270
312 6
2014 1566
1615 1176
1216 795
817 383
418 40
19 1692
1721 1291
1322 899
923 500
524 104
125 19
1827 1414
1428 1004
1029 604
630 206
231 20
1933 1501
1534 1107
1135 711
736 316
337 31
2039 1620
1640 1212
1241 817
842 418
443 25
44 1727
1746 1300
1347 905
948 497
549 91
150 17
1852 1423
1453 1027
1054 641
655 235
256 13
1958 1533
1559 1151
1160 767
761 392
362 56
2064 1684
1665 1275
1266 878
867 476
468 93
69 5
1771 1423
1372 1018
973 630
574 222
175 42
1877 1522
1478 1136
1079 729
680 321
281 38
1983 1643
1584 1225
1185 832
786 433
387 34
2089 1743

KeyboardInterrupt: 